In [13]:
repo_dir = "Repos"  
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-vgg19" / "src" / "vgg19"))
sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19")) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [15]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [16]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-04-10 17:33:29,571 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install


In [17]:
datasets = [
    '4-4116', # Selected images for training
    '4-411G', # Selected images for testing
    '2-7P5P', # Full multimodal dataset
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Multimodal workflow",
    workflow_type="Multimodal workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

2025-04-10 17:33:30,137 - WARNING - File /home/nguyent8/Repos/eye-ai-exec/notebooks/crop_multimodal_images.ipynb has been modified since last commit. Consider commiting before executing
2025-04-10 17:33:30,766 - INFO - Materialize bag 4-4116... 
2025-04-10 17:33:31,593 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-04-10 17:33:32,871 - INFO - Loading /data/4-4116_49ae0fbb8e51afdc463a0b95f4cfbaffb75c51364abfb4b0886b99506a46ac56/Dataset_4-4116
2025-04-10 17:33:33,858 - INFO - Creating new database for dataset: 4-4116 in /data/nguyent8/EyeAI_working/4-4116@334-RD2W-YM9J.db
2025-04-10 17:33:33,859 - INFO - Materialize bag 4-411G... 
2025-04-10 17:33:34,539 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-04-10 17:33:35,216 - INFO - Loading /data/4-411G_fa8115db0c4c767cd53098a2f7bad00c9a2ecd34eefb278c86e0435c5f67ad47/Dataset_4-411G
2025-04-10 17:33:35,734 - INFO - Creating new database for dataset: 4-411

In [18]:
print(execution)

caching_dir: /data
_working_dir: /data/nguyent8/EyeAI_working
execution_rid: 4-TWWC
workflow_rid: 4-TWQJ
asset_paths: {'Execution_Asset': [AssetFilePath('/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/downloaded-assets/Execution_Asset/optic_disk_crop_model.hdf5')]}
configuration: datasets=[DatasetSpec(rid='4-4116', materialize=True, version=DatasetVersion(major=2, minor=2, patch=0)), DatasetSpec(rid='4-411G', materialize=True, version=DatasetVersion(major=2, minor=2, patch=0)), DatasetSpec(rid='2-7P5P', materialize=True, version=DatasetVersion(major=2, minor=2, patch=0))] assets=['2-4JR6'] workflow=Workflow(name='Multimodal workflow', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/f6ae0e941f820286f7aca992390b8f0f3652a8e2--/notebooks/crop_multimodal_images.ipynb', workflow_type='Multimodal workflow', version=None, description='', rid=None, checksum='e69de29bb2d1d6434b8b29ae775ad8c2e48c5391') parameters={} description='Instance of cropping multimodal images.' argv=

In [19]:
training_ds_bag = execution.datasets[0]
testing_ds_bag = execution.datasets[1]
multimodal_full_ds_bag = execution.datasets[2]

crop_image_model =  execution.asset_paths["Execution_Asset"][0]

In [20]:
def get_dataframe_from_bag(ds_bag: DatasetBag, multimodal_full_ds_bag: DatasetBag):
    observation_table = ds_bag.get_table_as_dataframe('Observation')
    image_table = ds_bag.get_table_as_dataframe('Image')
    laterality_table = ds_bag.get_table_as_dataframe('Execution_Image_Fundus_Laterality')

    image_table_filtered = image_table[['RID', 'Filename', 'Observation']].rename(columns={'RID': 'RID_Image'})
    laterality_table_filtered = laterality_table[['Image', 'Image_Side']].rename(columns={'Image': 'RID_Image'})
    image_laterality = pd.merge(image_table_filtered, laterality_table_filtered, left_on='RID_Image', right_on='RID_Image', how='inner')
    observation_table_filtered = observation_table[['RID',  'Subject']].rename(columns={'RID': 'RID_Observation'})
    image_laterality_observation = pd.merge(image_laterality, observation_table_filtered, left_on='Observation', right_on='RID_Observation', how='inner')

    wide = EA.multimodal_wide(multimodal_full_ds_bag) 
    
    image_observation_laterality_subject_wide = pd.merge(
     wide, 
     image_laterality_observation, 
     left_on=['RID_Subject', 'Image_Side'], 
     right_on=['Subject', 'Image_Side'], 
     how='inner'
    )

    return image_observation_laterality_subject_wide

In [21]:
train_df = get_dataframe_from_bag(training_ds_bag, multimodal_full_ds_bag)
test_df= get_dataframe_from_bag(testing_ds_bag, multimodal_full_ds_bag)

/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)
/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)


In [31]:
from pathlib import Path

# Ensure working_dir is a Path object
working_dir = Path("~/working_dir")

# Save DataFrames as CSV files
train_csv_path = working_dir / "train.csv"
test_csv_path = working_dir / "test.csv"

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

In [23]:
table_name = 'Image'
EA.find_features(table_name)

[Feature(target_table=Image, feature_name=Annotation, feature_table=Annotation),
 Feature(target_table=Image, feature_name=Image_Diagnosis, feature_table=Image_Diagnosis),
 Feature(target_table=Image, feature_name=Fundus_Laterality, feature_table=Execution_Image_Fundus_Laterality),
 Feature(target_table=Image, feature_name=Fundus_Angle, feature_table=Execution_Image_Fundus_Angle)]

In [25]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [14]:
cl = [c for c in Feature.feature.feature_columns]
help(cl[0].type)

Help on Type in module deriva.core.ermrest_model object:

class Type(builtins.object)
 |  Type(type_doc)
 |  
 |  Named type.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, type_doc)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  prejson(self, prune=True)
 |  
 |  sqlite3_ddl(self) -> 'str'
 |      Return a SQLite3 column type DDL fragment for this type
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [30]:
output = execution._working_dir / "image_svg_output"
output.mkdir(parents= True, exist_ok=True)

In [ ]:
from vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/train.csv',
      '~/working_dir/output/output_train.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )
    
    preprocess_and_crop(
      multimodal_full_ds_bag,
      '~/working_dir/test.csv',
      '~/working_dir/output/output_test.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )

2025-04-10 17:37:24,599 - INFO - Start execution  ...
2025-04-10 17:37:24,882 - INFO - Execution Aborted
2025-04-10 17:37:25.310943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-10 17:37:25.722588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-10 17:37:25.726369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node ze

1/1 [==============================] - 2s 2s/step


2025-04-10 17:37:30,973 - INFO - Processing image /data/4-4116_49ae0fbb8e51afdc463a0b95f4cfbaffb75c51364abfb4b0886b99506a46ac56/Dataset_4-4116/data/asset/Image/2-660-IMAGE_0_(14).jpg : , trial 2, color channel green, resize function imgResize_primary


1/1 [==============================] - 0s 21ms/step


2025-04-10 17:37:31,830 - INFO - Processing image /data/4-4116_49ae0fbb8e51afdc463a0b95f4cfbaffb75c51364abfb4b0886b99506a46ac56/Dataset_4-4116/data/asset/Image/2-660-IMAGE_0_(14).jpg : , trial 3, color channel red, resize function imgResize_primary


...

2025-04-10 18:02:17,596 - INFO - Processing image /data/4-4116_49ae0fbb8e51afdc463a0b95f4cfbaffb75c51364abfb4b0886b99506a46ac56/Dataset_4-4116/data/asset/Image/198-656-IMAGE_0_(6).jpg : , trial 4, color channel blue, resize function imgResize_primary


1/1 [==============================] - 0s 21ms/step


2025-04-10 18:02:18,541 - INFO - Processing image /data/4-4116_49ae0fbb8e51afdc463a0b95f4cfbaffb75c51364abfb4b0886b99506a46ac56/Dataset_4-4116/data/asset/Image/198-656-IMAGE_0_(6).jpg : , trial 1, color channel grey, resize function imgResize_secondary


1/1 [==============================] - 0s 20ms/step


2025-04-10 18:02:20,148 - INFO - SVG for 2-BJKA saved.


In [36]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [40]:
image_rids = []
for file_name in os.listdir(output):
    image_rids.append(file_name.split("_")[1].split('.')[0])

In [41]:
csv_train = Path("~/working_dir/output/output_train.csv")
df = pd.read_csv(csv_train)
cropping_func_map_train = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [42]:
csv_test = Path("~/working_dir/output/output_test.csv")
df = pd.read_csv(csv_test)
cropping_func_map_test = dict(zip(df["Image RID"], df["Worked Image Cropping Function"]))

In [50]:
bounding_box_file_paths = []

for image_rid in image_rids:
    bounding_box_file = execution.asset_file_path(
        "Fundus_Bounding_Box", f"Cropped_{image_rid}.svg"
    )
    bounding_box_file_paths.append(bounding_box_file)

target_directory  = os.path.dirname(bounding_box_file_paths[0])

'/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/eye-ai/Fundus_Bounding_Box'

In [ ]:
import shutil 

def move_files(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        source_path = os.path.join(source_dir, filename)
        if os.path.isfile(source_path):
            target_path = os.path.join(target_dir, filename)
            shutil.move(source_path, target_path)
move_files(output, target_directory)

In [59]:
image_bounding_box_feature_list = []
for image_rid in image_rids:
    file_path = Path(target_directory) / f"Cropped_{image_rid}.svg"
    if file_path.exists():
        if image_rid in cropping_func_map_train:
            image_bounding_box_feature_list.append(
                ImageAnnotationFeature(
                    Image=image_rid,
                    Execution=execution.execution_rid,
                    Fundus_Bounding_Box=file_path,
                    Annotation_Function=cropping_func_map_train.get(image_rid),
                    Annotation_Type='Optic Nerve',
                ))
            
        if image_rid in cropping_func_map_test:
            image_bounding_box_feature_list.append(
                ImageAnnotationFeature(
                    Image=image_rid,
                    Execution=execution.execution_rid,
                    Fundus_Bounding_Box=file_path,
                    Annotation_Function=cropping_func_map_test.get(image_rid),
                    Annotation_Type='Optic Nerve',
                ))
        

In [66]:
image_bounding_box_feature_list[0]

ImageFeatureAnnotation(Execution='4-TWWC', Feature_Name='Annotation', Fundus_Bounding_Box=PosixPath('/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/eye-ai/Fundus_Bounding_Box/Cropped_2-BRVR.svg'), Annotation_Function='imgResize_secondary', Annotation_Type='Optic Nerve', Image='2-BRVR')

In [63]:
with execution.execute() as execution:
    execution.add_features(image_bounding_box_feature_list)

2025-04-11 14:10:16,009 - INFO - Start execution  ...
2025-04-11 14:10:16,148 - INFO - Successfully run Ml.
2025-04-11 14:10:16,222 - INFO - Algorithm execution ended.


In [ ]:
out = execution.upload_execution_outputs()

2025-04-11 14:11:59,641 - INFO - Uploading execution files...
2025-04-11 14:12:00,066 - INFO - Initializing uploader: GenericUploader v1.7.7 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-04-11 14:12:00,120 - INFO - Scanning files in directory [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset]...
2025-04-11 14:12:00,178 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/deriva-ml/Execution_Metadata/None/configuration.json].
2025-04-11 14:12:00,179 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/deriva-ml/Execution_Metadata/None/environment_snapshot_20250410_173347.txt].
2025-04-11 14:12:00,179 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/eye-ai/Fundus_Bounding_Box/Cropped_2-BRVR.svg].
2025-04-11 14:12:00,180 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-TWWC/asset/eye-ai/Fundus_Bounding_Box/